In [122]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict
from datetime import timedelta

In [123]:

# Cargar archivo Excel
file_path = "generic_input_case.xlsx"

# Leer cada hoja
df_horizonte = pd.read_excel(file_path, sheet_name="HORIZONTE")
df_bd_up = pd.read_excel(file_path, sheet_name="BD_UP")
df_frota = pd.read_excel(file_path, sheet_name="FROTA")
df_grua = pd.read_excel(file_path, sheet_name="GRUA")
df_fabrica = pd.read_excel(file_path, sheet_name="FABRICA")
df_rota = pd.read_excel(file_path, sheet_name="ROTA")

print(df_bd_up)
# Preprocesamiento básico
df_horizonte['FECHA'] = df_horizonte['DIA']

# df_fabrica["FECHA"] = pd.to_datetime(df_fabrica[["DIA", "MES"]].assign(ANO=2025))  # ajustar año si se requiere
df_bd_up["DATA_COLHEITA"] = pd.to_datetime(df_bd_up["DATA_COLHEITA"])
df_rota["FAZENDA"] = df_rota["Fazenda"]
# print (df_horizonte)

# Crear listas útiles
dias = df_horizonte["FECHA"].tolist()
print( dias)
transportadores = df_frota["TRANSPORTADOR"].unique().tolist()

        UP                      FAZENDA          DB    VOLUME       RSP  \
0   S6C421                      SIRIEMA  406.666302   4500.00  1.289003   
1   S3AX03                   INDIANA II  410.081889   2124.07  1.302226   
2   S6C297                    FORTALEZA  408.376272   4791.00  1.367399   
3   S5AW14  PIRACEMA_GLEBA PULADOR - DX  467.553351  18533.08  1.378530   
4   S5AW09  PIRACEMA_GLEBA PULADOR - DX  485.300233   3320.00  1.403137   
5   S5AW13  PIRACEMA_GLEBA PULADOR - DX  471.883265  16280.54  1.410163   
6   S5AW05  PIRACEMA_GLEBA PULADOR - DX  450.000000   4960.00  1.440000   
7   S6BX01         SANTO ANTONIO SALIG.  450.000000  12700.00  1.440000   
8   S5AW10  PIRACEMA_GLEBA PULADOR - DX  489.657814  12776.12  1.441440   
9   S5AK10              TURVO III (LEX)  494.987423  14000.00  1.443982   
10  S5AK05              TURVO III (LEX)  493.203247   9830.26  1.479045   
11  S6C334                    FORTALEZA  458.550818   5974.00  1.505206   
12  S3AX04               

c:\Users\wilma\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [ ]:
def construir_solucion_inicial():
    plan = []  # lista de decisiones diarias: una fila por camión asignado
    up_disponibles = df_bd_up[["UP","FAZENDA","RSP", "DB", "VOLUME"]].copy()
    up_disponibles["RESTANTE"] = up_disponibles["VOLUME"]

    for fecha in dias:
        demanda_dia = df_fabrica[df_fabrica["DIA"] == fecha]
        # if demanda_dia.empty:
        #     continue

        demanda_min = demanda_dia["DEMANDA_MIN"].values[0]
        demanda_max = demanda_dia["DEMANDA_MAX"].values[0]
        # print(demanda_min)
        volumen_trans = 0
        fazenda_act=  {t: None for t in transportadores}    # Control de fazenda activa por transportador

        # Asignar camiones por transportador
        for t in transportadores:
            f_dia = df_frota[(df_frota["TRANSPORTADOR"] == t)]
            if f_dia.empty:
                continue
            f_min, f_max = int(f_dia["FROTA_MIN"]), int(f_dia["FROTA_MAX"])
            if f_max == 0:
                continue

            ass_c=random.randint(f_min,f_max)
            gruas=df_grua[df_grua["TRANSPORTADOR"]==t].iloc[0]["QTD_GRUAS"]
            porc_min=df_grua[df_grua["TRANSPORTADOR"]==t].iloc[0]["PORCENTAGEM_VEICULOS_MIN"]
            rutas_t = df_rota[df_rota["TRANSPORTADOR"] == t]

            # r_i=rutas_t.iloc[random.randint(0,len(rutas_t)-1)]["FAZENDA"]
            # idx_gruas=random.sample(rutas_t[rutas_t["FAZENDA"] == r_i].index.tolist(),k=gruas)
            rutas_t = rutas_t.sample(frac = 1).sort_values(by=["FAZENDA"]).iloc[:gruas]
            print(rutas_t)

            # Asignar camiones a UPs viables
            for _, ruta in rutas_t.iterrows():

                up = ruta["ORIGEM"]
                if up not in up_disponibles["UP"].values:
                    continue

                up_data = up_disponibles[up_disponibles["UP"] == up]#.iloc[0]
                rest=up_data.iloc[0]["RESTANTE"]
                # print(up)
                # print(up_data)
                # print(rest)
                carga = ruta["CAIXA_CARGA"]
                ciclos = ruta["TEMPO_CICLO"] #máximo número de viajes en 1 día
                # max_cams = int(8 // ciclos)  # simplificación: 8 horas por día de operación
                asignar = min(int(porc_min*f_max), rest // (carga*ciclos))
                asignar =max(asignar,ass_c-asignar)

                if asignar == 0:
                    continue

                volumen_dia = asignar * carga *ciclos

                if volumen_trans + volumen_dia > demanda_max:
                    continue  # evitar exceder

                plan.append({
                    "UP": up,
                    "FAZENDA": up_data["FAZENDA"],
                    "TRANSPORTADOR": t,
                    "FECHA": fecha,
                    "DB": up_data["DB"],
                    "RSP": up_data["RSP"],
                    "QTD_VEICULOS": asignar,
                    "VOLUME": volumen_dia
                })

                volumen_trans += volumen_dia
                up_disponibles.loc[up_disponibles["UP"] == up, "RESTANTE"] -= volumen_dia
                # print("check")
                break  # asignado un UP por transportador ese día

        # Si no se logró cumplir la demanda mínima, se penaliza (o se anota para revisión)
        if not (volumen_trans > demanda_min) and (volumen_trans < demanda_max):
            print(f"⚠️ Demanda no cubierta en {fecha}: {demanda_min} < {volumen_trans:.1f} < {demanda_max}")
    print(plan)
    return pd.DataFrame(plan)

print("🔨 Construyendo solución inicial...")
plan_inicial = construir_solucion_inicial()


🔨 Construyendo solución inicial...
    ORIGEM DESTINO TRANSPORTADOR  CAIXA_CARGA  TEMPO_CICLO  CICLO_LENTO  \
14  S5AK05     LIM       Pastori           66          1.4            0   
23  S5AW05     LIM       Pastori           66          1.5            0   

                        Fazenda                      FAZENDA  
14              TURVO III (LEX)              TURVO III (LEX)  
23  PIRACEMA_GLEBA PULADOR - DX  PIRACEMA_GLEBA PULADOR - DX  
    ORIGEM DESTINO TRANSPORTADOR  CAIXA_CARGA  TEMPO_CICLO  CICLO_LENTO  \
35  S6C297     LIM         Tover           66          2.1            0   
30  S6BG11     LIM         Tover           66          2.5            0   

      Fazenda    FAZENDA  
35  FORTALEZA  FORTALEZA  
30    SINIMBU    SINIMBU  
    ORIGEM DESTINO TRANSPORTADOR  CAIXA_CARGA  TEMPO_CICLO  CICLO_LENTO  \
4   S3AX06     LIM       Rampazo           66          1.8            0   
11  S5AK08     LIM       Rampazo           66          1.4            0   

            Fazen

C:\Users\wilma\AppData\Local\Temp\ipykernel_36272\1391118767.py:22: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  f_min, f_max = int(f_dia["FROTA_MIN"]), int(f_dia["FROTA_MAX"])
C:\Users\wilma\AppData\Local\Temp\ipykernel_36272\1391118767.py:22: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  f_min, f_max = int(f_dia["FROTA_MIN"]), int(f_dia["FROTA_MAX"])
C:\Users\wilma\AppData\Local\Temp\ipykernel_36272\1391118767.py:22: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  f_min, f_max = int(f_dia["FROTA_MIN"]), int(f_dia["FROTA_MAX"])
C:\Users\wilma\AppData\Local\Temp\ipykernel_36272\1391118767.py:22: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use

    ORIGEM DESTINO TRANSPORTADOR  CAIXA_CARGA  TEMPO_CICLO  CICLO_LENTO  \
32  S6BG13     LIM         Tover           66          2.5            0   
34  S6BX01     LIM         Tover           66          2.2            0   

                 Fazenda               FAZENDA  
32               SINIMBU               SINIMBU  
34  SANTO ANTONIO SALIG.  SANTO ANTONIO SALIG.  
    ORIGEM DESTINO TRANSPORTADOR  CAIXA_CARGA  TEMPO_CICLO  CICLO_LENTO  \
13  S5AK10     LIM       Rampazo           66          1.4            0   
20  S5AW10     LIM       Rampazo           66          1.5            0   

                        Fazenda                      FAZENDA  
13              TURVO III (LEX)              TURVO III (LEX)  
20  PIRACEMA_GLEBA PULADOR - DX  PIRACEMA_GLEBA PULADOR - DX  
⚠️ Demanda no cubierta en 19: 6400 < 4072.2 < 8000
    ORIGEM DESTINO TRANSPORTADOR  CAIXA_CARGA  TEMPO_CICLO  CICLO_LENTO  \
16  S5AK09     LIM       Pastori           66          1.4            0   
27  S5AW14 

C:\Users\wilma\AppData\Local\Temp\ipykernel_36272\1391118767.py:22: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  f_min, f_max = int(f_dia["FROTA_MIN"]), int(f_dia["FROTA_MAX"])
C:\Users\wilma\AppData\Local\Temp\ipykernel_36272\1391118767.py:22: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  f_min, f_max = int(f_dia["FROTA_MIN"]), int(f_dia["FROTA_MAX"])
C:\Users\wilma\AppData\Local\Temp\ipykernel_36272\1391118767.py:22: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  f_min, f_max = int(f_dia["FROTA_MIN"]), int(f_dia["FROTA_MAX"])
C:\Users\wilma\AppData\Local\Temp\ipykernel_36272\1391118767.py:22: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use

In [ ]:
def calcular_delta_db(df_plan):
    df_por_dia = df_plan.groupby("FECHA").apply(
        lambda g: np.average(g["DB"], weights=g["VOLUME"])
    ).reset_index(name="DB_MEDIA")

    delta = df_por_dia["DB_MEDIA"].max() - df_por_dia["DB_MEDIA"].min()
    return delta

def guardar_solucion(df_plan, nombre="solucion_constructiva.xlsx"):
    df_plan["DIA"] = df_plan["FECHA"].dt.day
    df_plan["MES"] = df_plan["FECHA"].dt.month
    columnas = ["UP", "FAZENDA", "TRANSPORTADOR", "DIA", "MES", "DB", "RSP", "QTD_VEICULOS", "VOLUME"]
    df_plan[columnas].to_excel(nombre, index=False)


In [ ]:
def es_factible(plan, verbose=False):
    factible = True
    
    # 1. Validar volumen y RSP diarios
    for fecha in plan["FECHA"].unique():
        plan_dia = plan[plan["FECHA"] == fecha]
        total_vol = plan_dia["VOLUME"].sum()
        
        fila_fab = df_fabrica[df_fabrica["FECHA"] == fecha]
        if fila_fab.empty:
            continue
        dmin, dmax = fila_fab["DEMANDA_MIN"].values[0], fila_fab["DEMANDA_MAX"].values[0]
        
        if not (dmin <= total_vol <= dmax):
            if verbose: print(f"❌ Volumen fuera de rango en {fecha}: {total_vol}")
            factible = False

        rsp_media = np.average(plan_dia["RSP"], weights=plan_dia["VOLUME"])
        rmin, rmax = fila_fab["RSP_MIN"].values[0], fila_fab["RSP_MAX"].values[0]
        if not (rmin <= rsp_media <= rmax):
            if verbose: print(f"❌ RSP fuera de rango en {fecha}: {rsp_media:.2f}")
            factible = False

    # 2. Validar flota diaria por transportador
    for (fecha, t), sub in plan.groupby(["FECHA", "TRANSPORTADOR"]):
        fila = df_frota[(df_frota["TRANSPORTADOR"] == t) & (df_frota["DIA"] == fecha.day)]
        if fila.empty:
            continue
        fmin, fmax = int(fila["FROTA_MIN"]), int(fila["FROTA_MAX"])
        total_vehs = sub["QTD_VEICULOS"].sum()
        if not (fmin <= total_vehs <= fmax):
            if verbose: print(f"❌ Flota fuera de rango en {fecha} para {t}: {total_vehs}")
            factible = False

    # 3. Validar grúas y porcentaje mínimo de vehículos por UP
    for (fecha, t), sub in plan.groupby(["FECHA", "TRANSPORTADOR"]):
        fila_grua = df_grua[df_grua["TRANSPORTADOR"] == t]
        if fila_grua.empty:
            continue
        gruas = int(fila_grua["QTD_GRUAS"])
        por_min = float(fila_grua["PORCENTAGEM_VEICULOS_MIN"])
        if sub["UP"].nunique() > gruas:
            if verbose: print(f"❌ Exceso de UPs para {t} en {fecha}")
            factible = False
        total_vehs = sub["QTD_VEICULOS"].sum()
        for _, fila in sub.iterrows():
            porc = fila["QTD_VEICULOS"] / total_vehs
            if porc < por_min:
                if verbose: print(f"❌ Porcentaje mínimo no cumplido en {fecha} para UP {fila['UP']}")
                factible = False

    # 4. Validar fazenda única por transportador
    for (fecha, t), sub in plan.groupby(["FECHA", "TRANSPORTADOR"]):
        if sub["FAZENDA"].nunique() > 1:
            if verbose: print(f"❌ Más de una fazenda atendida por {t} en {fecha}")
            factible = False

    # 5. Validar transporte completo para UPs < 7000
    volumenes = df_bd_up.set_index("UP")["VOLUME"].to_dict()
    for up, sub in plan.groupby("UP"):
        if volumenes[up] < 7000:
            total = sub["VOLUME"].sum()
            esperado = volumenes[up]
            if abs(total - esperado) > 1e-3:
                if verbose: print(f"❌ UP {up} <7000 no fue despachada completamente (total={total})")
                factible = False
            fechas = sorted(sub["FECHA"].dt.date.unique())
            if len(fechas) > 1:
                if verbose: print(f"❌ UP {up} <7000 fue despachada en más de un día")
                factible = False
        else:
            fechas = sorted(sub["FECHA"].dt.date.unique())
            if len(fechas) > 2:
                if verbose: print(f"❌ UP {up} >7000 tiene más de dos días de despacho")
                factible = False

    return factible


In [ ]:
# if __name__ == "__main__":
print("🔨 Construyendo solución inicial...")
plan_inicial = construir_solucion_inicial()

print(f"📝 Se generaron {len(plan_inicial)} asignaciones.")

print("\n🔎 Validando factibilidad...")
factible = es_factible(plan_inicial, verbose=True)

if factible:
    print("✅ ¡La solución inicial es factible!")
else:
    print("❌ La solución inicial NO es factible.")

delta_db = calcular_delta_db(plan_inicial)
print(f"📉 Objetivo (ΔDB): {delta_db:.4f}")

guardar_solucion(plan_inicial, "solucion_inicial.xlsx")

🔨 Construyendo solución inicial...
⚠️ Demanda no cubierta en 2: 6400 < 5062.2 < 8000
⚠️ Demanda no cubierta en 3: 6400 < 805.2 < 8000
⚠️ Demanda no cubierta en 4: 6400 < -138.6 < 8000
⚠️ Demanda no cubierta en 5: 6400 < 897.6 < 8000
⚠️ Demanda no cubierta en 6: 6400 < -871.2 < 8000
⚠️ Demanda no cubierta en 7: 6400 < 1603.8 < 8000
⚠️ Demanda no cubierta en 8: 6400 < -2145.0 < 8000
⚠️ Demanda no cubierta en 9: 6400 < 1980.0 < 8000
⚠️ Demanda no cubierta en 10: 6400 < -1815.0 < 8000
⚠️ Demanda no cubierta en 11: 6400 < 1980.0 < 8000
⚠️ Demanda no cubierta en 12: 6400 < -2145.0 < 8000
⚠️ Demanda no cubierta en 13: 6400 < 1980.0 < 8000
⚠️ Demanda no cubierta en 14: 6400 < -1815.0 < 8000
⚠️ Demanda no cubierta en 15: 6400 < 1980.0 < 8000
⚠️ Demanda no cubierta en 16: 6400 < -2145.0 < 8000
⚠️ Demanda no cubierta en 17: 6400 < 1980.0 < 8000
⚠️ Demanda no cubierta en 18: 6400 < -1815.0 < 8000
⚠️ Demanda no cubierta en 19: 6400 < 1980.0 < 8000
⚠️ Demanda no cubierta en 20: 6400 < -2145.0 < 8000

C:\Users\wilma\AppData\Local\Temp\ipykernel_36272\1408350081.py:21: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  f_min, f_max = int(f_dia["FROTA_MIN"]), int(f_dia["FROTA_MAX"])


KeyError: 'FECHA'